<a href="https://colab.research.google.com/github/kashperova/ssl-hsi-course-work/blob/main/notebooks/supervised_ca_ffdn_subset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
!git clone https://@github.com/kashperova/ssl-hsi-course-work.git

Cloning into 'ssl-hsi-course-work'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 43 (delta 1), reused 43 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (43/43), 38.97 MiB | 17.71 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [1]:
%cd ssl-hsi-course-work/src

/content/ssl-hsi-course-work/src


In [6]:
!curl -LsSf https://astral.sh/uv/install.sh | sh

downloading uv 0.7.8 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [ ]:
!uv pip install --system scikit-learn==1.6.0
!uv pip install --system seaborn==0.13.0
!uv pip install --system matplotlib==3.10.1
!uv pip install --system plotly==6.0.0
!uv pip install --system torchmetrics==1.7.2
!uv pip install --system scipy==1.15.2

In [2]:
from models.ca_ffdn import CA_FFDN
from modules.trainers.supervised import BaseSupervisedTrainer
from modules.datasets.hsi import HyperspectralDataset
from config.train_config import BaseTrainConfig
from utils.seed import set_seed
from utils.metrics import Metrics, Task
from utils.data import load_hsi_dataset, get_stratified_subset

import os
import wandb
import torch.optim as optim
from torch.utils.data import Subset, Dataset

from torch import nn

In [9]:
os.environ["WANDB_API_KEY"] = ""

In [3]:
wandb.init()

wandb: Currently logged in as: skashperova (kashperova-test) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
set_seed(42)

Random seed set to 42


In [5]:
class TrainConfig(BaseTrainConfig):
    epochs: int = 50
    train_batch_size: int = 64
    eval_batch_size: int = 64
    train_test_split: float = 0.7

In [6]:
patches, labels = load_hsi_dataset(
    dataset_name="IndianPines", root_dir="../data", pca_components=30, patch_size=15
)

In [7]:
dataset = HyperspectralDataset(patches, labels)
dataset10 = get_stratified_subset(dataset, 0.2)
dataset20 = get_stratified_subset(dataset, 0.1)

In [14]:
def train(subset: Subset, num_classes: int, save_name: str):
    model = CA_FFDN(in_channels=30, num_classes=num_classes)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5)
    criterion = nn.CrossEntropyLoss()

    trainer = BaseSupervisedTrainer(
        model=model,
        optimizer=optimizer,
        lr_scheduler=scheduler,
        dataset=subset,
        config=TrainConfig(),
        metrics=Metrics(
            task=Task.MULTICLASS_CLASSIFICATION,
            num_classes=num_classes,
            average="micro",
        ),
        loss_fn=criterion,
        save_dir="/content/gdrive/MyDrive/[kpi] ssl course work",
        save_name=save_name,
    )
    trainer.train(verbose=False)


def reinit_wandb():
    wandb.finish(quiet=True)
    wandb.init()

In [9]:
train(dataset20, 16, "ca_ffdn_20_ip")

Training: 100%|██████████| 50/50 [00:31<00:00,  1.60it/s]


In [ ]:
reinit_wandb()

In [11]:
train(dataset10, 16, "ca_ffdn_10_ip")

Training: 100%|██████████| 50/50 [00:51<00:00,  1.03s/it]


In [12]:
patches, labels = load_hsi_dataset(
    dataset_name="PaviaUniversity", root_dir="../data", pca_components=30, patch_size=15
)
dataset = HyperspectralDataset(patches, labels)
dataset10 = get_stratified_subset(dataset, 0.2)
dataset20 = get_stratified_subset(dataset, 0.1)

In [ ]:
reinit_wandb()

In [16]:
train(dataset20, 9, "ca_ffdn_20_pu")

Training: 100%|██████████| 50/50 [01:41<00:00,  2.02s/it]


In [ ]:
reinit_wandb()

In [18]:
train(dataset10, 9, "ca_ffdn_10_pu")

Training: 100%|██████████| 50/50 [03:20<00:00,  4.01s/it]


In [20]:
patches, labels = load_hsi_dataset(
    dataset_name="KSC", root_dir="../data", pca_components=30, patch_size=15
)
dataset = HyperspectralDataset(patches, labels)
dataset10 = get_stratified_subset(dataset, 0.2)
dataset20 = get_stratified_subset(dataset, 0.1)

In [ ]:
reinit_wandb()

In [22]:
train(dataset20, 13, "ca_ffdn_20_ksc")

Training: 100%|██████████| 50/50 [00:13<00:00,  3.79it/s]


In [ ]:
reinit_wandb()

In [24]:
train(dataset10, 13, "ca_ffdn_10_ksc")

Training: 100%|██████████| 50/50 [00:25<00:00,  1.92it/s]
